<a href="https://colab.research.google.com/github/Andres54ml/ME02_especializacion/blob/main/PREPROCESAMIETO/Preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descarga de datos históricos de Bancolombia ADR (CIB)

En esta celda se utiliza la librería `yfinance` para descargar el historial de precios de apertura del ADR de Bancolombia (símbolo "CIB") desde el 11 de mayo de 2015 hasta el 9 de mayo de 2025. Luego, se filtra únicamente la columna **Open** (precio de apertura) y se reinicia el índice para que la columna de fechas quede como una columna regular del DataFrame.


In [ ]:
import yfinance as yf

# Descarga histórico de Bancolombia ADR (CIB)
df_principal = yf.download(
    "CIB",
    start="2015-05-11",
    end="2025-05-9"
)

# Filtrar solo la columna Open y mover el índice Date a columna
df_principal = df_principal[['Open']].reset_index()

print(df_principal.head())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price        Date       Open
Ticker                   CIB
0      2015-05-11  22.255103
1      2015-05-12  22.114088
2      2015-05-13  22.293994
3      2015-05-14  22.396111
4      2015-05-15  22.279408


## Carga y procesamiento de datos externos

En esta celda se carga un archivo de Excel que contiene una hoja llamada **"Datos"**. Todos los valores se leen como texto (`dtype=str`). Luego:

1. Se convierte la columna **Fecha** al formato de fecha (`datetime`), especificando que el día viene primero (formato DD/MM/YYYY).
2. Finalmente, se renombra la columna **Fecha** a **Date** para mantener consistencia con otros DataFrames.

In [ ]:
import pandas as pd

# Lee las dos hojas como texto
archivo = "/content/graficador_series (1) (1).xlsx"
sheets = pd.read_excel(
    archivo,
    sheet_name=["Datos"],
    dtype=str
)

df_secundario =  sheets["Datos"]

# 1) Convertir a datetime, indicando dayfirst para DD/MM/YYYY
df_secundario['Fecha'] = pd.to_datetime(
    df_secundario['Fecha'],
    format="%d/%m/%Y",
    dayfirst=True,
    errors="raise"
)

# 2) (Opcional) Si además quieres renombrar la columna a 'Date'
df_secundario.rename(columns={'Fecha': 'Date'}, inplace=True)


## Unificación de columnas y combinación de DataFrames

- Se renombran las columnas del DataFrame `df_principal` uniendo los niveles con guiones bajos (`_`) en caso de que existan múltiples niveles (MultiIndex).
- Luego se asegura que el índice esté reseteado (aunque ya se hizo anteriormente).
- Se imprime el número de niveles de columnas tanto para `df_principal` como para `df_secundario`.
- Finalmente, se realiza un *merge* (unión) entre `df_principal` y `df_secundario` usando la columna **Date** como clave. Se usa una unión tipo *left join*, conservando todos los registros de `df_principal` y anexando los de `df_secundario` cuando coincidan las fechas.

In [ ]:
# Por ejemplo, unir con guión bajo los niveles que haya
df_principal.columns = [
    "_".join(filter(None, col)).strip()
    for col in df_principal.columns.values
]

df_principal = df_principal.reset_index()

# print("Niveles de columnas en principal:", df_principal.columns.nlevels)
# print("Niveles de columnas en secundario:", df_secundario.columns.nlevels)

df_merged = df_principal.merge(
    df_secundario,
    on='Date',
    how='left',
    suffixes=('', '_sec')
)

## Limpieza de datos: manejo de valores faltantes

- Se seleccionan algunas columnas económicas relevantes del DataFrame combinado `df_merged`.
- En esas columnas, se reemplazan los valores "-" por `NaN` para que pandas los reconozca como datos faltantes.
- Finalmente, se imprime la cantidad de valores nulos (`NaN`) en cada una de las columnas seleccionadas, como verificación del reemplazo.

In [ ]:
import numpy as np

cols = [
    "M1, mensual(Dato fin de mes)",
    "Inflación total(Dato fin de mes)",
    "Crecimiento PIB real, Trimestral, base: 2015, Ajuste estacional(Dato fin de trimestre)",
    "Tasa de desempleo - total nacional(Dato fin de mes)"
]

# Reemplaza todos los "-" por NaN solo en esas columnas
df_merged[cols] = df_merged[cols].replace("-", np.nan)

# Comprueba que ahora pandas los reconoce como NaN
print(df_merged[cols].isna().sum())


M1, mensual(Dato fin de mes)                                                              2430
Inflación total(Dato fin de mes)                                                          2429
Crecimiento PIB real, Trimestral, base: 2015, Ajuste estacional(Dato fin de trimestre)    2489
Tasa de desempleo - total nacional(Dato fin de mes)                                       2430
dtype: int64


# Imputación de valores faltantes en series temporales macroeconómicas

Este notebook implementa un flujo completo para limpiar e imputar valores faltantes en variables macroeconómicas de frecuencia mensual y trimestral.

---

## Objetivo

Transformar y completar series de tiempo con valores faltantes (representados como guiones), aplicando técnicas adecuadas de imputación según la naturaleza de cada variable.

---

## Paso 1: Limpieza de datos

- Se reemplazan guiones (`-`, `–`, `—`) por `NaN`, que representan valores faltantes.
- Se reemplazan las comas (`,`) por puntos (`.`) para estandarizar el formato decimal.

---

## Paso 2: Conversión a tipo numérico

- Las columnas seleccionadas se convierten a tipo numérico (`float`).
- Cualquier valor no convertible se convierte automáticamente a `NaN`.

---

## Paso 3: Manejo de fechas

- Se convierte la columna `Date` a formato `datetime` si existe.
- Se establece como índice del DataFrame para facilitar la imputación temporal.
- Se ordena cronológicamente el índice.

---

## Paso 3.5: Verificación previa

- Se imprime el número de valores nulos por columna antes de la imputación.
- Se visualizan las primeras filas para inspección rápida del estado de los datos.

---

## Paso 4: Imputación de valores faltantes

- Para variables con comportamiento suave y continuo (como M1 y desempleo), se usa interpolación temporal (`interpolate(method='time')`) y backfill.
- Para variables que cambian de forma más discreta (como inflación y PIB), se usa forward-fill (`fillna(method='ffill')`) seguido de backfill si es necesario.

---


In [ ]:
import numpy as np
import pandas as pd

cols = [
    'M1, mensual(Dato fin de mes)',
    'Inflación total(Dato fin de mes)',
    'Crecimiento PIB real, Trimestral, base: 2015, Ajuste estacional(Dato fin de trimestre)',
    'Tasa de desempleo - total nacional(Dato fin de mes)'
]

# --- Paso 1: Limpieza de guiones y conversión de comas a puntos ---
# Esto reemplaza posibles representaciones de valores faltantes y ajusta el separador decimal.
print("Starting data cleaning and conversion...")
df_merged[cols] = (
    df_merged[cols]
    .replace(r'^\s*[-–—]\s*$', np.nan, regex=True) # Reemplaza guiones (cortos, medios, largos) rodeados de espacios por NaN
    .replace(',', '.', regex=True) # Reemplaza comas por puntos como separador decimal
)
print("Cleaning and comma replacement complete.")

# --- Paso 2: Conversión a float ---
# Convierte las columnas a tipo numérico (float), forzando los errores a NaN.
print("Converting columns to numeric...")
df_merged[cols] = df_merged[cols].apply(pd.to_numeric, errors='coerce')
print("Numeric conversion complete.")

# --- Paso 3: Asegurar que la información temporal está en datetime y establecer el índice ---
# Convierte la columna o el índice de fecha a tipo datetime y establece el índice si es necesario.
print("Ensuring datetime index and sorting...")
if 'Date' in df_merged.columns:
    # Si 'Date' existe como columna, la convertimos y la establecemos como índice
    df_merged['Date'] = pd.to_datetime(df_merged['Date'], errors='coerce')
    df_merged = df_merged.set_index('Date')
    # Eliminar la columna Date original si ya se convirtió a índice
    # if 'Date' in df_merged.columns:
    #     df_merged = df_merged.drop(columns=['Date']) # Esto es opcional dependiendo de si Date duplicada
else:
    # Si la fecha ya está en el índice, solo aseguramos la conversión a datetime
    df_merged.index = pd.to_datetime(df_merged.index, errors='coerce')

# Ordenar el DataFrame por índice de tiempo
df_merged = df_merged.sort_index()
print("Datetime index set and sorted.")

# --- Paso 3.5: Verificación de NaN después de la limpieza y antes de la imputación ---
# Imprime el número de NaN por columna y las primeras filas para inspeccionar los datos antes de imputar.
print("\n--- Verification Before Imputation ---")
print("NaN counts per column after cleaning and conversion:")
print(df_merged[cols].isnull().sum())
print("\nHead of DataFrame (first 10 rows) before imputation:")
print(df_merged[cols].head(10))
print("------------------------------------\n")


# --- Paso 4: Imputaciones ---
# Rellenar los valores faltantes.
# Usamos interpolate(method='time') seguido de fillna(method='bfill') para las series mensuales/diarias.
# Usamos fillna(method='ffill') seguido de fillna(method='bfill') para las series trimestrales o donde ffill es más apropiado.
# El .fillna(method='bfill') después de la imputación principal se encarga de los NaNs iniciales
# (usando el primer valor válido encontrado hacia adelante) y los NaNs finales.
print("Starting imputation...")

# Series mensuales/diarias: M1, Tasa de Desempleo (interpolación basada en tiempo, luego bfill para iniciales/finales)
df_merged['M1, mensual(Dato fin de mes)'] = df_merged['M1, mensual(Dato fin de mes)'].interpolate(method='time').fillna(method='bfill')
df_merged['Tasa de desempleo - total nacional(Dato fin de mes)'] = df_merged['Tasa de desempleo - total nacional(Dato fin de mes)'].interpolate(method='time').fillna(method='bfill')

# Series mensuales/trimestrales: Inflación, Crecimiento PIB (ffill, luego bfill para iniciales/finales)
df_merged['Inflación total(Dato fin de mes)'] = df_merged['Inflación total(Dato fin de mes)'].fillna(method='ffill').fillna(method='bfill')
df_merged['Crecimiento PIB real, Trimestral, base: 2015, Ajuste estacional(Dato fin de trimestre)'] = (
    df_merged['Crecimiento PIB real, Trimestral, base: 2015, Ajuste estacional(Dato fin de trimestre)']
    .fillna(method='ffill')
    .fillna(method='bfill')
)

print("Imputation complete.")

# --- Paso 5: Verificación Final ---
# Vuelve a verificar si quedan NaNs y muestra las primeras filas después de la imputación.
print("\n--- Final Verification After Imputation ---")
print("NaN counts per column after imputation:")
print(df_merged[cols].isnull().sum())
print("\nHead of DataFrame (first 10 rows) after imputation:")
print(df_merged[cols].head(10))
print("------------------------------------")

Starting data cleaning and conversion...
Cleaning and comma replacement complete.
Converting columns to numeric...
Numeric conversion complete.
Ensuring datetime index and sorting...
Datetime index set and sorted.

--- Verification Before Imputation ---
NaN counts per column after cleaning and conversion:
M1, mensual(Dato fin de mes)                                                              2430
Inflación total(Dato fin de mes)                                                          2429
Crecimiento PIB real, Trimestral, base: 2015, Ajuste estacional(Dato fin de trimestre)    2489
Tasa de desempleo - total nacional(Dato fin de mes)                                       2430
dtype: int64

Head of DataFrame (first 10 rows) before imputation:
            M1, mensual(Dato fin de mes)  Inflación total(Dato fin de mes)  \
Date                                                                         
2015-05-11                           NaN                               NaN   
2015-05-12  

<ipython-input-6-0a3dad6b073a>:64: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged['M1, mensual(Dato fin de mes)'] = df_merged['M1, mensual(Dato fin de mes)'].interpolate(method='time').fillna(method='bfill')
<ipython-input-6-0a3dad6b073a>:65: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged['Tasa de desempleo - total nacional(Dato fin de mes)'] = df_merged['Tasa de desempleo - total nacional(Dato fin de mes)'].interpolate(method='time').fillna(method='bfill')
<ipython-input-6-0a3dad6b073a>:68: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged['Inflación total(Dato fin de mes)'] = df_merged['Inflación total(Dato fin de mes)'].fillna(method='ffill').fillna(method='bfill')
<ipython-input-6-0a3dad6b073a>:7

## Paso 2: Escalado de datos

- Se combinan las columnas de entrada y la columna objetivo para aplicar un escalado conjunto.
- Se utiliza `MinMaxScaler` para escalar los valores entre 0 y 1.
- Esto asegura que todas las variables estén en la misma escala, lo cual es importante para el entrenamiento de redes neuronales.

---

## Paso 3: Separación de datos escalados

- Se identifican los índices de las variables de entrada y del objetivo dentro del array escalado.
- Se separan los datos escalados en:
  - `training_set_scaled`: datos de entrada multivariados.
  - `y_data_scaled`: columna objetivo ya escalada.

---

## Paso 4: Construcción de secuencias para entrenamiento

- Se utiliza una ventana deslizante de tamaño `window_size` para construir las secuencias de entrada.
- Cada entrada (`X_train`) es una secuencia de longitud fija con múltiples variables.
- Cada etiqueta (`y_train`) es el valor de la variable objetivo inmediatamente después de la ventana.

---

## Paso 5: Conversión a arrays NumPy

- Las listas `X_train` y `y_train` se convierten en arrays NumPy.
- Esto es necesario para alimentar estos datos al modelo RNN.

---

## Verificación final

- Se imprime la forma de `X_train` y `y_train` para confirmar que el formato es el adecuado para el entrenamiento.
  - `X_train`: (número de secuencias, tamaño de ventana, número de variables).
  - `y_train`: (número de secuencias,).

---

Este preprocesamiento deja los datos listos para ser usados como entrada en una red LSTM o GRU multivariada.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# --- Define las columnas ---
input_cols = [
    'Open_CIB',
    # 'M1, mensual(Dato fin de mes)',
    'Inflación total(Dato fin de mes)'
    # Agrega aquí otras columnas si las deseas usar como input
]

target_col = 'Open_CIB'  # Columna objetivo (puedes cambiarla)

# --- Preparación de los datos ---
columns_to_scale = input_cols + ([target_col] if target_col not in input_cols else [])
data_to_scale = df_merged[columns_to_scale].values  # Asegúrate de que df_merged esté definido

# --- Escalado ---
print("Starting data scaling...")
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_to_scale)
print("Data scaling complete.")

# Identificación de índices
input_indices = [columns_to_scale.index(col) for col in input_cols]
target_index = columns_to_scale.index(target_col)

training_set_scaled = scaled_data[:, input_indices]
y_data_scaled = scaled_data[:, target_index]

print("\n--- Scaling Results Summary ---")
print("Original shape of data used for scaling:", data_to_scale.shape)
print("Shape of the scaled data array:", scaled_data.shape)
print("Shape of the scaled input features (training_set_scaled):", training_set_scaled.shape)
print("Shape of the scaled target variable (y_data_scaled):", y_data_scaled.shape)
print("-------------------------------\n")

# --- Construcción de secuencias (X) y etiquetas (y) ---
window_size = 60
X_train = []
y_train = []

data_length = len(training_set_scaled)
for i in range(window_size, data_length):
    X_train.append(training_set_scaled[i - window_size:i, :])
    y_train.append(y_data_scaled[i])

X_train = np.array(X_train)
y_train = np.array(y_train)

print("\n--- Multivariate Windowing Results (after scaling) ---")
print(f"Window Size: {window_size}")
print("X_train shape (Sequences):", X_train.shape)
print("y_train shape (Labels):", y_train.shape)
print("----------------------------------------------------")

# --- Guardar datos como CSV ---
df_scaled = pd.DataFrame(scaled_data, columns=columns_to_scale)
df_scaled.to_csv("datos_escalados.csv", index=False)

df_y_train = pd.DataFrame(y_train, columns=["Target_scaled"])
df_y_train.to_csv("y_train.csv", index=False)

X_train_flat = X_train.reshape(X_train.shape[0], -1)  # (N, window, features) → (N, window * features)
df_X_train = pd.DataFrame(X_train_flat)
df_X_train.to_csv("X_train_flat.csv", index=False)

print("\n✅ Datos guardados como CSV: 'datos_escalados.csv', 'y_train.csv' y 'X_train_flat.csv'")


Starting data scaling...
Data scaling complete.

--- Scaling Results Summary ---
Original shape of data used for scaling: (2515, 2)
Shape of the scaled data array: (2515, 2)
Shape of the scaled input features (training_set_scaled): (2515, 2)
Shape of the scaled target variable (y_data_scaled): (2515,)
-------------------------------


--- Multivariate Windowing Results (after scaling) ---
Window Size: 60
X_train shape (Sequences): (2455, 60, 2)
y_train shape (Labels): (2455,)
----------------------------------------------------

✅ Datos guardados como CSV: 'datos_escalados.csv', 'y_train.csv' y 'X_train_flat.csv'
